<a href="https://colab.research.google.com/github/DCI-alxogm/ml2023-Isaias-Hernandez/blob/main/proyecto/proyecto_IHL_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto -
Por Isaías Hernández Lares - Machine learning 2023

# Content:
1. Carga de información

* Importarción de librerías
* Carga de datasets
2. Revisión de la información

* Estadística descriptiva
* Valores faltantes
3. Análisis exploratorio

* Verificar la distribución de las clases
* Verificar la distribución de las características
* Verificar como características numéricas se relacionan con las clases
4. Preparación del input de información

* Limpieza de datos
* Separación en X y Y
* Entrenamiento, validación y pueba
5. Construcción del modelo

* Entrenamiento del modelo
* Predicción del modelo
* Evaluación del modelo
6. Mejoramiento del modelo

* Mejora de hiperparametros
* Guardar el modelo final